In [1]:
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import random

In [2]:
kafka_topic_name = "Topic3"
kafka_bootstrap_servers = 'localhost:9092'

spark = SparkSession \
        .builder \
        .appName("Encryption_Decryption_Streaming ") \
        .master("local[*]") \
        .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1") \
        .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

In [3]:
df_transaction = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
        .option("subscribe", kafka_topic_name) \
        .option("startingOffsets", "earliest") \
        .load()

In [4]:
df_transaction.describe()

DataFrame[summary: string, topic: string, partition: string, offset: string, timestampType: string]

In [5]:
df1_transaction = df_transaction.selectExpr("CAST(value AS STRING)", "timestamp")

df_schema_string = "encrypted_account_number STRING,lcy_amount STRING"



df2_transaction= df1_transaction \
        .select(from_csv(col("value"), df_schema_string) \
                .alias("transaction"), "timestamp")

df3_transaction = df2_transaction.select("transaction.*", "timestamp")
df4_transaction = df3_transaction.withColumn('account_number',
                               expr("trim(aes_decrypt(unbase64(encrypted_account_number), '1234567890abcdef', 'ECB', 'PKCS'))")).select('account_number','lcy_amount')
    
df4_transaction.createOrReplaceTempView("transaction");
final_transaction = spark.sql("SELECT * FROM transaction")
Transaction_agg_write_stream = final_transaction \
        .writeStream \
        .trigger(processingTime='5 seconds') \
        .outputMode("append") \
        .option("truncate", "false") \
        .format("memory") \
        .queryName("Transaction_Table") \
        .start()
Transaction_agg_write_stream.awaitTermination(1)

False

In [8]:
df = spark.sql("SELECT * FROM Transaction_Table")
df.show(10)

+-------------------+----------+
|     account_number|lcy_amount|
+-------------------+----------+
|02XYZXYZ10017529992|      NULL|
|02XYZXYZ10017529992|      NULL|
|02XYZXYZ10017517823|      NULL|
|02XYZXYZ10017517823|      NULL|
|02XYZXYZ10017517823|      NULL|
|02XYZXYZ10017519116|      NULL|
|02XYZXYZ10017519121|      NULL|
|02XYZXYZ10017520752|      NULL|
|02XYZXYZ10017520752|      NULL|
|02XYZXYZ10017520752|      NULL|
+-------------------+----------+
only showing top 10 rows

